In [4]:
import googlemaps
import os
from dotenv import load_dotenv

load_dotenv()

gmaps = googlemaps.Client(key=os.getenv("GOOGLE_MAPS_API_KEY"))

### Places

In [32]:
# this call takes any natrual language, uses google's super secret giga search algo
place_result = gmaps.places('boba spots near UC Irvine')

for place in place_result['results']:
    print(place['name'])

Sharetea - UCI
Cha For Tea - University Center
Pink Pig
TP TEA – Irvine Main
MACU TEA - Irvine
One Zo Boba - Irvine
Jam Jam Tea Lab - Irvine
Tea Maru
OMOMO TEA SHOPPE
Bopomofo Cafe
Presotea Irvine
Boba Square
The Alley Irvine
Tastea Irvine
Boba Guys Costa Mesa
heytea (Irvine Jeffrey)
Wushiland Boba - Irvine
Cha For Tea - Woodbridge Village Center
Sunright Tea Studio - Irvine
BAKO TEA HOUSE


In [33]:
# place result is a dictionary with these keys (I think for our project only results useful)
place_result.keys()

dict_keys(['html_attributions', 'next_page_token', 'results', 'status'])

In [34]:
# results is an array of dictionaries with these keys 
results = place_result["results"][0]
place_result["results"][0].keys()

dict_keys(['business_status', 'formatted_address', 'geometry', 'icon', 'icon_background_color', 'icon_mask_base_uri', 'name', 'opening_hours', 'photos', 'place_id', 'price_level', 'rating', 'reference', 'types', 'user_ratings_total'])

In [35]:
print("Is it open now? ", results.get("opening_hours", {}).get("open_now", "Unknown"))
print("Business stats:", results.get("business_status", "Unknown"))
print("Price level:", results.get("price_level", "Unknown"))
print("user ratings total:", results.get("user_ratings_total", "Unknown"))
print("Rating: ", results.get("rating", "Unknown"))

Is it open now?  True
Business stats: OPERATIONAL
Price level: 1
user ratings total: 223
Rating:  4.3


In [37]:
print("Every place open now: ")

for i in place_result["results"]:
    is_open = i.get("opening_hours", {}).get("open_now", False)
    print(i["name"], i["rating"], ", ", is_open)

Every place open now: 
Sharetea - UCI 4.3 ,  True
Cha For Tea - University Center 4.3 ,  True
Pink Pig 3.9 ,  True
TP TEA – Irvine Main 4.5 ,  True
MACU TEA - Irvine 4 ,  True
One Zo Boba - Irvine 4.4 ,  True
Jam Jam Tea Lab - Irvine 4.6 ,  True
Tea Maru 3.8 ,  True
OMOMO TEA SHOPPE 4.3 ,  True
Bopomofo Cafe 4.4 ,  True
Presotea Irvine 3.9 ,  True
Boba Square 4 ,  True
The Alley Irvine 4.1 ,  True
Tastea Irvine 4.4 ,  True
Boba Guys Costa Mesa 4.3 ,  True
heytea (Irvine Jeffrey) 4.5 ,  True
Wushiland Boba - Irvine 4.3 ,  True
Cha For Tea - Woodbridge Village Center 4.2 ,  True
Sunright Tea Studio - Irvine 4.3 ,  True
BAKO TEA HOUSE 4.5 ,  True


In [42]:
sorted_places = [] 
for i in place_result["results"]:
    if i.get("opening_hours", {}).get("open_now", False):
        sorted_places.append((i["name"], i.get("rating", 0)))
sorted_places.sort(key=lambda k: k[1], reverse=True)

In [44]:
print("Top 3 spots by rating: ")
for i in range(3):
    print(sorted_places[i])

Top 3 spots by rating: 
('Jam Jam Tea Lab - Irvine', 4.6)
('TP TEA – Irvine Main', 4.5)
('heytea (Irvine Jeffrey)', 4.5)


In [49]:
print("Top 3 by Bayesian rating: ")

C = sum(all_ratings) / len(all_ratings)
m = 50  # minimum threshold

bayes = []
for i in place_result["results"]:
    if i.get("opening_hours", {}).get("open_now", False):
        rating = i.get("rating", 0)
        n = i.get("user_ratings_total", 0)
        sum_R = rating * n  # ΣR
        score = (C * m + sum_R) / (m + n)
        bayes.append((i["name"], score, rating, n))
bayes.sort(key=lambda k: k[1], reverse=True)


for i in range(len(bayes)):
    print(bayes[i])

Top 3 by Bayesian rating: 
('Jam Jam Tea Lab - Irvine', 4.502777777777778, 4.6, 130)
('heytea (Irvine Jeffrey)', 4.482294617563739, 4.5, 656)
('TP TEA – Irvine Main', 4.453007518796992, 4.5, 216)
('BAKO TEA HOUSE', 4.438118811881188, 4.5, 152)
('One Zo Boba - Irvine', 4.388057324840765, 4.4, 578)
('Tastea Irvine', 4.37, 4.4, 200)
('Bopomofo Cafe', 4.321052631578947, 4.4, 45)
('Cha For Tea - University Center', 4.295840266222961, 4.3, 551)
('OMOMO TEA SHOPPE', 4.295711835334477, 4.3, 533)
('Sunright Tea Studio - Irvine', 4.2921875, 4.3, 270)
('Sharetea - UCI', 4.290842490842492, 4.3, 223)
('Boba Guys Costa Mesa', 4.283974358974358, 4.3, 106)
('Wushiland Boba - Irvine', 4.276635514018692, 4.3, 57)
('Cha For Tea - Woodbridge Village Center', 4.208116883116883, 4.2, 258)
('The Alley Irvine', 4.118987341772152, 4.1, 345)
('Boba Square', 4.091240875912408, 4, 87)
('MACU TEA - Irvine', 4.083333333333333, 4, 100)
('Pink Pig', 4.065094339622641, 3.9, 56)
('Presotea Irvine', 4.057657657657657, 3

In [52]:
import json

results_to_save = []
for place in place_result["results"]:
    results_to_save.append({
        "name": place.get("name"),
        "open_now": place.get("opening_hours", {}).get("open_now"),
        "price_level": place.get("price_level"),
        "rating": place.get("rating"),
        "user_ratings_total": place.get("user_ratings_total")
    })

with open("places_data.json", "w") as f:
    json.dump(results_to_save, f, indent=2)

### Distance